In [1]:
import math
import pandas as pd
import numpy as np

class InoculumPreparation(object):
    
    def __init__(self,BioreactorWV,BioreactorChanges):
        self.BioreactorWV = BioreactorWV
        self.BioreactorChanges = BioreactorChanges
        
    def Sizing(self):
        #Load Equipment from Database
        Equipment=pd.read_excel('Database.xlsx',sheetname='Equipment',index_col=0)
        #Load Materials from Database
        Materials=pd.read_excel('Database.xlsx',sheetname='Materials',index_col=0)
        #Load SOPs from Database
        Protocol=pd.read_excel('Database.xlsx',sheetname='SOPs',index_col=0)
        #Calculate seed train
        SeedBioreactorsWVList=[]
        CorrectedSeedBioreactorList=[]
        i=1
        while i<=self.BioreactorChanges+1:
            j=self.BioreactorChanges+2
            if self.BioreactorWV*(Protocol.get_value('Inoculation Ratio','Seed Bioreactors')**(j-i))/Protocol.get_value('Space Efficiency','Seed Bioreactors')<Equipment.get_value('Bioreactor','Min. Size'):
                SeedBioreactorWV=0
                CorrectedSeedBioreactor=0
            else:
                SeedBioreactorWV=self.BioreactorWV*(Protocol.get_value('Inoculation Ratio','Seed Bioreactors')**(j-i))
                SeedBioreactor=SeedBioreactorWV/Protocol.get_value('Space Efficiency','Seed Bioreactors')
                if SeedBioreactor<=10:
                    CorrectedSeedBioreactor=math.ceil(SeedBioreactor)
                elif SeedBioreactor<=100 and SeedBioreactor>10:
                    CorrectedSeedBioreactor=int(math.ceil(SeedBioreactor/10.0))*10
                elif SeedBioreactor<=1000 and SeedBioreactor>100:
                    CorrectedSeedBioreactor=int(math.ceil(SeedBioreactor/100.0))*100
                else:
                    CorrectedSeedBioreactor=int(math.ceil(SeedBioreactor/1000.0))*1000
            i=i+1
            SeedBioreactorsWVList.append(SeedBioreactorWV)
            CorrectedSeedBioreactorList.append(CorrectedSeedBioreactor)
        #cell culture media volume
        CCMediaVolumeList=[]
        for x in SeedBioreactorsWVList:
            CCMediaVolume=x*(1-Protocol.get_value('Inoculation Ratio','Seed Bioreactors'))*(1+Protocol.get_value('Media Overfill','Hold-Tanks'))
            CCMediaVolumeList.append(CCMediaVolume)
        TotalMediaVolume=sum(CCMediaVolumeList)
        #Shake flasks TODO
        
        return SeedBioreactorsWVList,CorrectedSeedBioreactorList,CCMediaVolumeList,TotalMediaVolume
    
    #timeframe, equip and materials, cost breakdown TODO
    def Timeframe(self):
        #Load SOPs from Database
        Protocol=pd.read_excel('Database.xlsx',sheetname='SOPs',index_col=0)
        #time to prepare for a batch
        PrepTime=Protocol.get_value('Preparation Time (hr)','Seed Bioreactors')+Protocol.get_value('Load Time (hr)','Seed Bioreactors')
        #time to operate a batch
        OperTime=Protocol.get_value('Operation Time (hr)','Seed Bioreactors')
        #time to clean
        CIPTime=Protocol.get_value('CIP Time (hr)','Seed Bioreactors')+Protocol.get_value('SIP Time (hr)','Seed Bioreactors')
        #time to turnaround bioreactors for next batch
        TurnAroundTime=Protocol.get_value('Turnaround Time (hr)','Seed Bioreactors')
        #time to turnaround hold-tanks
        HoldTanksCIPandSIPTime=Protocol.get_value('CIP Time (hr)','Hold-Tanks')+Protocol.get_value('SIP Time (hr)','Hold-Tanks')
        return PrepTime,OperTime,CIPTime,TurnAroundTime,HoldTanksCIPandSIPTime
    
    def EquipmentAndMaterials(self):
        #Load Equipment from Database
        Equipment=pd.read_excel('Database.xlsx',sheetname='Equipment',index_col=0)
        #Load Materials from Database
        Materials=pd.read_excel('Database.xlsx',sheetname='Materials',index_col=0)
        #Load SOPs from Database
        Protocol=pd.read_excel('Database.xlsx',sheetname='SOPs',index_col=0)
        #CC Media hold-tank with GF, PW, WFI, CIP
        CCMediaCost=InoculumPreparation.Sizing(self)[3]*Materials.get_value('Cell Culture Media','Base Cost')
        MediaHoldTankCostList=[]
        MediaHoldTankGuardList=[]
        MediaCIPBufferList=[]
        MediaCIPBufferCostList=[]
        MediaWFIVolumeList=[]
        MediaPWVolumeList=[]
        MediaWFIandPWCostList=[]
        for x in InoculumPreparation.Sizing(self)[2]:
            if x/Protocol.get_value('Space Efficiency','Hold-Tanks')>Equipment.get_value('Hold-Tank','Max. Size'):
                NumberOfMediaTanks=math.ceil((x/Protocol.get_value('Space Efficiency','Hold-Tanks'))/Equipment.get_value('Hold-Tank','Max. Size'))
            else:
                NumberOfMediaTanks=1
            MediaTankVolume=(x/Protocol.get_value('Space Efficiency','Hold-Tanks'))/NumberOfMediaTanks
            if MediaTankVolume<=10:
                CorrectedMediaTankVolume=math.ceil(MediaTankVolume)
            elif MediaTankVolume<=100 and MediaTankVolume>10:
                CorrectedMediaTankVolume=int(math.ceil(MediaTankVolume/10.0))*10
            elif MediaTankVolume<=1000 and MediaTankVolume>100:
                CorrectedMediaTankVolume=int(math.ceil(MediaTankVolume/100.0))*100
            else:
                CorrectedMediaTankVolume=int(math.ceil(MediaTankVolume/1000.0))*1000
            MediaHoldTankCost=NumberOfMediaTanks*Equipment.get_value('Hold-Tank','Base Cost')*(CorrectedMediaTankVolume/Equipment.get_value('Hold-Tank','Base Size'))**Equipment.get_value('Hold-Tank','Scaling Factor')
            MediaHoldTankGuard=NumberOfMediaTanks*Materials.get_value('Hold-Tank Guard Filter','Base Cost')*(CorrectedMediaTankVolume/Materials.get_value('Hold-Tank Guard Filter','Base Size'))**Materials.get_value('Hold-Tank Guard Filter','Scaling Factor')
            MediaCIPBuffer=NumberOfMediaTanks*(Protocol.get_value('Acid Buffer','Hold-Tanks')+Protocol.get_value('Caustic Buffer','Hold-Tanks'))*CorrectedMediaTankVolume*(1+Protocol.get_value('Buffer Overfill','Hold-Tanks'))
            MediaCIPBufferCost=MediaCIPBuffer*Materials.get_value('CIP Buffer','Base Cost')
            MediaHoldTankDiameter=NumberOfMediaTanks*(CorrectedMediaTankVolume*4000/(math.pi*Protocol.get_value('H/D tank ratio','Hold-Tanks')))**(1/3)
            MediaWFIVolume=(1+Protocol.get_value('Buffer Overfill','Hold-Tanks'))*MediaHoldTankDiameter/100*Protocol.get_value('Water Flowrate (L/hr/m)','Hold-Tanks')/60*Protocol.get_value('WFI Rinse (min)','Hold-Tanks')
            MediaPWVolume=(1+Protocol.get_value('Buffer Overfill','Hold-Tanks'))*MediaHoldTankDiameter/100*Protocol.get_value('Water Flowrate (L/hr/m)','Hold-Tanks')/60*Protocol.get_value('PW Rinse (min)','Hold-Tanks')
            MediaWFIandPWCost=MediaWFIVolume*Materials.get_value('WFI','Base Cost')+MediaPWVolume*Materials.get_value('PW','Base Cost')
            MediaHoldTankCostList.append(MediaHoldTankCost)
            MediaHoldTankGuardList.append(MediaHoldTankGuard)
            MediaCIPBufferList.append(MediaCIPBuffer)
            MediaCIPBufferCostList.append(MediaCIPBufferCost)
            MediaWFIVolumeList.append(MediaWFIVolume)
            MediaPWVolumeList.append(MediaPWVolume)
            MediaWFIandPWCostList.append(MediaWFIandPWCost)
        TotalMediaHoldTanksCost=sum(MediaHoldTankCostList)
        TotalMediaHoldGFCost=sum(MediaHoldTankGuardList)
        TotalMediaHoldCIPVolume=sum(MediaCIPBufferList)
        TotalMediaHoldCIPCost=sum(MediaCIPBufferCostList)
        TotalMediaHoldWFIVolume=sum(MediaWFIVolumeList)
        TotalMediaHoldPWVolume=sum(MediaPWVolumeList)
        TotalMediaHoldWFIandPWCost=sum(MediaWFIandPWCostList)
        #Seed Bioreactors with PW, WFI, CIP
        SeedBioreactorCostList=[]
        SeedBioreactorCIPBufferList=[]
        SeedBioreactorCIPBufferCostList=[]
        SeedBioreactorWFIVolumeList=[]
        SeedBioreactorPWVolumeList=[]
        SeedBioreactorWFIandPWCostList=[]
        for x in InoculumPreparation.Sizing(self)[1]:
            SeedBioreactorCost=Equipment.get_value('Bioreactor','Base Cost')*(x/Equipment.get_value('Bioreactor','Base Size'))**Equipment.get_value('Bioreactor','Scaling Factor')
            SeedBioreactorCIPBuffer=(Protocol.get_value('Acid Buffer','Bioreactor')+Protocol.get_value('Caustic Buffer','Bioreactor'))*x*(1+Protocol.get_value('Buffer Overfill','Hold-Tanks'))
            SeedBioreactorCIPBufferCost=SeedBioreactorCIPBuffer*Materials.get_value('CIP Buffer','Base Cost')
            SeedBioreactorDiameter=(x*4000/(math.pi*Protocol.get_value('H/D tank ratio','Bioreactor')))**(1/3)
            SeedBioreactorWFIVolume=(1+Protocol.get_value('Buffer Overfill','Hold-Tanks'))*SeedBioreactorDiameter/100*Protocol.get_value('Water Flowrate (L/hr/m)','Bioreactor')/60*Protocol.get_value('WFI Rinse (min)','Bioreactor')
            SeedBioreactorPWVolume=(1+Protocol.get_value('Buffer Overfill','Hold-Tanks'))*SeedBioreactorDiameter/100*Protocol.get_value('Water Flowrate (L/hr/m)','Bioreactor')/60*Protocol.get_value('PW Rinse (min)','Bioreactor')
            SeedBioreactorWFIandPWCost=SeedBioreactorWFIVolume*Materials.get_value('WFI','Base Cost')+SeedBioreactorPWVolume*Materials.get_value('PW','Base Cost')
            SeedBioreactorCostList.append(SeedBioreactorCost)
            SeedBioreactorCIPBufferList.append(SeedBioreactorCIPBuffer)
            SeedBioreactorCIPBufferCostList.append(SeedBioreactorCIPBufferCost)
            SeedBioreactorWFIVolumeList.append(SeedBioreactorWFIVolume)
            SeedBioreactorPWVolumeList.append(SeedBioreactorPWVolume)
            SeedBioreactorWFIandPWCostList.append(SeedBioreactorWFIandPWCost)
        TotalSeedBioreactorsCost=sum(SeedBioreactorCostList)
        TotalSeedCIPVolume=sum(SeedBioreactorCIPBufferList)
        TotalSeedCIPCost=sum(SeedBioreactorCIPBufferCostList)
        TotalSeedWFIVolume=sum(SeedBioreactorWFIVolumeList)
        TotalSeedPWVolume=sum(SeedBioreactorPWVolumeList)
        TotalSeedWFIandPWCost=sum(SeedBioreactorWFIandPWCostList)
        return TotalSeedBioreactorsCost,TotalMediaHoldTanksCost,CCMediaCost,TotalMediaHoldCIPCost,TotalSeedCIPCost,TotalMediaHoldWFIandPWCost,TotalSeedWFIandPWCost,TotalMediaHoldGFCost
    
    def CostBreakdown(self):
        EquipmentPurchaseCost=sum(InoculumPreparation.EquipmentAndMaterials(self)[0:2])
        #costs per batch
        ChemicalReagentsCost=sum(InoculumPreparation.EquipmentAndMaterials(self)[2:7])
        ConsumablesCost=sum(InoculumPreparation.EquipmentAndMaterials(self)[7:8])
        LabourCost=0
        return LabourCost,ChemicalReagentsCost,ConsumablesCost,EquipmentPurchaseCost


In [2]:
InoculationTrain=InoculumPreparation(BioreactorWV=10000,BioreactorChanges=3)
print(InoculationTrain.Sizing())
print(InoculationTrain.EquipmentAndMaterials())
print(InoculationTrain.CostBreakdown())

([0, 10.000000000000002, 100.00000000000001, 1000.0], [0, 20, 200, 2000], [0.0, 10.350000000000001, 103.50000000000001, 1035.0], 1148.8499999999999)
(920178.37169432151, 107872.76419032773, 57442.499999999993, 7658.9999999999991, 7658.9999999999991, 543.003299039289, 474.3570743280855, 601.70994009888193)
(0, 73777.860373367366, 601.70994009888193, 1028051.1358846492)
